In [1]:
import requests
import pandas as pd
import urllib3
import time
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Login

In [2]:
def get_with_auth(url, login, password):
    # Define the parameters to send in the GET request
    params = {
        'username': login,
        'password': password
    }

    try:
        # Send GET request with parameters
        response = requests.get(url, params=params, verify=False ) #, cert=("C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/cert.pem", "C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/key.pem"))
        
        # Raise an exception if the request failed
        response.raise_for_status()
        
        # Return the response content
        return response  # Use response.json() for JSON data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

In [3]:
base_url = 'https://10.250.115.39:4000/'
    
login_url = "LOGIN"

URL = base_url + login_url

content = get_with_auth(URL, 'Web', 'web')

token = content.json()['Token']

token

'F85CC2E269BB41EB891B95D82E5EE314'

## Patients

In [4]:
def get_patients(base_url, token):
    # Define the URL for getting patients
    url = base_url + "/GET/patients"

    # Define the headers with the API token
    headers = {
        'API-token': token
    }

    try:
        # Send GET request to fetch patients
        response = requests.get(url, headers=headers, verify=False)
        
        # Raise an exception if the request failed
        response.raise_for_status()
        
        # Return the response content
        return response  # Assuming the response is in JSON format
    except requests.exceptions.RequestException as e:
        print(f"Error fetching patients: {e}")
        return None

url = base_url + "/GET/patients"


response = get_patients(base_url, token)

if response is not None:
    # Convert the response to a DataFrame
    patients = response.json()
    df_patients = pd.DataFrame(patients['Patients'])



In [5]:
df_patients.head()

,PatientIDx,PatientID,FirstName,LastName,DateOfBirth
0,NEXTGEN_43622.7870662732,1111,first,test,NaN
1,PC10T4L72760_43623.4196205208,9999,Jane,Doe,NaN
2,NEXTGEN_43622.6655321528,0000,cam,test,NaN
3,PC10T4L77647_43623.5757282639,send mail to Martin,End,Tuesday,NaN
4,PC10T4L790165_43625.4810629630,62.399,Amanda Alves,Pollone,1983.12.01


# Treatments

In [6]:
def get_treatments(base_url, token, patientIDx):
    """
    Retrieve treatments for a given patientIDx from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patientIDx (str): The unique patient identifier.

    Returns:
        dict or None: The JSON response containing treatments, or None if request fails.
    """
    url = f"{base_url}/GET/TREATMENT"
    params = {'patientIDx': patientIDx}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching treatments: {e}")
        return None

In [7]:
df_patients.sample(5, random_state=42)  # Display a sample of 5 patients for verification

,PatientIDx,PatientID,FirstName,LastName,DateOfBirth
1558,PC1R85KM_45119.4351552199,520.124,"CARNEIRO, ZENI",07/08/1981,1981.08.07
1165,PC10T4L7_44803.5316606366,141.725,"FERRARI, CAROLINE T.",21/09/1977,1977.09.21
2577,PC1R85KM_45814.4062967361,884.770,"GUIMARAES, FLORA",22/12/83,1983.12.22
422,PC10T4L72052_44120.4346775926,73.644,"GLEZER, TATIANA",13/07/1977,1977.07.01
695,PC10T4L7_44391.4511898148,45.645,"MATSUO, MARIANA A.",22/07/1982,1982.07.22


In [8]:
get_treatments(base_url, token, 'PC10T4L7_44803.5316606366')

{'TreatmentList': ['2022 - 1449']}

# Embryo ID

In [9]:
def get_embryo_id(base_url, token, patientIDx, treatment_name):
    """
    Retrieve embryo ID data for a given patientIDx and treatment name from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patientIDx (str): The unique patient identifier.
        treatment_name (str): The treatment name.

    Returns:
        dict or None: The JSON response containing embryo ID data, or None if request fails.
    """
    url = f"{base_url}/GET/embryoID"
    params = {
        'PatientIDx': patientIDx,
        'TreatmentName': treatment_name
    }
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo ID: {e}")
        return None

In [10]:
get_embryo_id(base_url, token, 'PC10T4L7_44803.5316606366', '2022 - 1449')

{'EmbryoIDList': ['D2022.08.30_S01790_I3253_P-1',
  'D2022.08.30_S01790_I3253_P-2',
  'D2022.08.30_S01790_I3253_P-3',
  'D2022.08.30_S01790_I3253_P-4',
  'D2022.08.30_S01790_I3253_P-5',
  'D2022.08.30_S01790_I3253_P-6',
  'D2022.08.30_S01790_I3253_P-7',
  'D2022.08.30_S01790_I3253_P-8',
  'D2022.08.30_S01790_I3253_P-9',
  'D2022.08.30_S01790_I3253_P-10']}

# Fertilization time

In [11]:
def get_fertilization_time(base_url, token, embryo_id):
    """
    Retrieve fertilization time for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing fertilization time, or None if request fails.
    """
    url = f"{base_url}/GET/fertilizationtime"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching fertilization time: {e}")
        return None


In [12]:
# Example usage:
get_fertilization_time(base_url, token, 'D2022.08.30_S01790_I3253_P-1')

{'FertilizationTime': '2022.08.30 12:35:00'}

# Image runs

In [13]:
def get_image_runs(base_url, token, embryo_id):
    """
    Retrieve image runs for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing image runs, or None if request fails.
    """
    url = f"{base_url}/GET/imageruns"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching image runs: {e}")
        return None


In [14]:

# Example usage:
get_image_runs(base_url, token, 'D2022.08.30_S01790_I3253_P-1')

{'ImageRuns': [{'Run': 1, 'Time': 0.3},
  {'Run': 2, 'Time': 0.4},
  {'Run': 3, 'Time': 0.7},
  {'Run': 4, 'Time': 0.9},
  {'Run': 5, 'Time': 1.1},
  {'Run': 6, 'Time': 1.3},
  {'Run': 7, 'Time': 1.4},
  {'Run': 8, 'Time': 1.6},
  {'Run': 9, 'Time': 1.8},
  {'Run': 10, 'Time': 2},
  {'Run': 11, 'Time': 2.1},
  {'Run': 12, 'Time': 2.3},
  {'Run': 13, 'Time': 2.7},
  {'Run': 14, 'Time': 3},
  {'Run': 15, 'Time': 3.6},
  {'Run': 16, 'Time': 4.6},
  {'Run': 17, 'Time': 4.8},
  {'Run': 18, 'Time': 5},
  {'Run': 19, 'Time': 5.2},
  {'Run': 20, 'Time': 5.3},
  {'Run': 21, 'Time': 5.5},
  {'Run': 22, 'Time': 5.7},
  {'Run': 23, 'Time': 5.9},
  {'Run': 24, 'Time': 6},
  {'Run': 25, 'Time': 6.2},
  {'Run': 26, 'Time': 6.4},
  {'Run': 27, 'Time': 6.6},
  {'Run': 28, 'Time': 6.7},
  {'Run': 29, 'Time': 6.9},
  {'Run': 30, 'Time': 7.1},
  {'Run': 31, 'Time': 7.3},
  {'Run': 32, 'Time': 7.4},
  {'Run': 33, 'Time': 7.6},
  {'Run': 34, 'Time': 7.8},
  {'Run': 35, 'Time': 8},
  {'Run': 36, 'Time': 8.1}

# Evaluation - KID score

In [15]:
def get_evaluation(base_url, token, embryo_id):
    """
    Retrieve evaluation data for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing evaluation data, or None if request fails.
    """
    url = f"{base_url}/GET/evaluation"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching evaluation: {e}")
        return None

In [16]:
get_evaluation(base_url, token, 'D2022.08.30_S01790_I3253_P-1')

{'Model': 'KIDScoreD5 v3',
 'Evaluation': '9.3',
 'User': 'ADMIN',
 'EvaluationDate': '2022.09.28'}

# Emrbyo Fate

In [17]:
def get_embryo_fate(base_url, token, embryo_id):
    """
    Retrieve embryo fate data for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing embryo fate data, or None if request fails.
    """
    url = f"{base_url}/GET/embryofate"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo fate: {e}")
        return None


In [18]:

# Example usage:
get_embryo_fate(base_url, token, 'D2022.08.30_S01790_I3253_P-1')

{'EmbryoFate': 'Freeze'}

# Embryo Details

In [19]:
def get_embryo_details(base_url, token, embryo_id):
    """
    Retrieve detailed embryo information for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing embryo details, or None if request fails.
    """
    url = f"{base_url}/GET/embryodetails"
    params = {'EmbryoID': embryo_id}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo details: {e}")
        return None


In [20]:

# Example usage:
get_embryo_details(base_url, token, 'D2022.08.30_S01790_I3253_P-1')

{'InstrumentNumber': 3253,
 'Position': 4,
 'WellNumber': 1,
 'FertilizationTime': '2022.08.30 12:35:00',
 'FertilizationMethod': 'ICSI',
 'EmbryoFate': 'Freeze',
 'EmbryoDescriptionID': 'AA1'}

# Embryo Data

In [21]:
def get_embryo_data(base_url, token, patient_idx, treatment_name):
    """
    Retrieve embryo data for a given PatientIDx and TreatmentName from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patient_idx (str): The unique patient identifier.
        treatment_name (str): The treatment name.

    Returns:
        dict or None: The JSON response containing embryo data, or None if request fails.
    """
    url = f"{base_url}/GET/embryodata"
    params = {
        'PatientIDx': patient_idx,
        'TreatmentName': treatment_name
    }
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching embryo data: {e}")
        return None

In [22]:
get_embryo_data(base_url, token, 'PC10T4L7_44803.5316606366', '2022 - 1449')

{'EmbryoDataList': [{'EmbryoID': 'D2022.08.30_S01790_I3253_P-1',
   'EmbryoDetails': {'InstrumentNumber': 3253,
    'Position': 4,
    'WellNumber': 1,
    'FertilizationTime': '2022.08.30 12:35:00',
    'FertilizationMethod': 'ICSI',
    'EmbryoFate': 'Freeze',
    'EmbryoDescriptionID': 'AA1'},
   'AnnotationList': [{'Name': 'tPB2',
     'Time': 3.6,
     'Value': '44803.6736659028',
     'Timestamp': '2022.08.30 16:10:04'},
    {'Name': 'tPNa',
     'Time': 4.6,
     'Value': '44803.716606875',
     'Timestamp': '2022.08.30 17:11:54'},
    {'Name': 'tPNf',
     'Time': 21.1,
     'Value': '44804.4042601157',
     'Timestamp': '2022.08.31 09:42:08'},
    {'Name': 't2',
     'Time': 23.3,
     'Value': '44804.4952783796',
     'Timestamp': '2022.08.31 11:53:12'},
    {'Name': 't3',
     'Time': 32.7,
     'Value': '44804.8883459607',
     'Timestamp': '2022.08.31 21:19:13'},
    {'Name': 't4',
     'Time': 32.9,
     'Value': '44804.895961331',
     'Timestamp': '2022.08.31 21:30:11'}

# Transfer

In [23]:
def get_transfers(base_url, token, patient_idx):
    """
    Retrieve transfer data for a given PatientIDx from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        patient_idx (str): The unique patient identifier.

    Returns:
        dict or None: The JSON response containing transfer data, or None if request fails.
    """
    url = f"{base_url}/GET/transfers"
    params = {'PatientIDx': patient_idx}
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, params=params, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching transfers: {e}")
        return None


In [24]:

# Example usage:
get_transfers(base_url, token, 'PC10T4L7_44803.5316606366')

Error fetching transfers: Expecting value: line 1 column 1 (char 0)


                       EmbryoID  Viability  Time Version            Timestamp
0  D2019.06.19_S00028_I3253_P-1       1.03   115   2.0.4  2024.04.30 07:55:37
1  D2019.06.19_S00028_I3253_P-2       1.05   115   2.0.4  2024.04.30 07:55:35
2  D2019.06.19_S00028_I3253_P-3       1.06   115   2.0.4  2024.04.30 07:55:33
3  D2019.06.19_S00028_I3253_P-4       1.01   115   2.0.4  2024.04.30 07:56:06
4  D2019.07.03_S00054_I3253_P-2       3.93   120   2.0.4  2024.05.28 15:20:00


In [ ]:
def get_idascore(base_url, token):
    """
    Retrieve IDASCORE data for a given EmbryoID from the API.

    Args:
        base_url (str): The base URL of the API.
        token (str): The API authentication token.
        embryo_id (str): The EmbryoID.

    Returns:
        dict or None: The JSON response containing IDASCORE data, or None if request fails.
    """
    url = f"{base_url}/GET/IDASCORE"
    headers = {'API-token': token}

    try:
        response = requests.get(url, headers=headers, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching IDASCORE: {e}")
        return None

In [ ]:
# Call the function to get IDASCORE data
idascore_data = get_idascore(base_url, token)

# Check if data was retrieved successfully and 'Scores' key exists
if idascore_data and 'Scores' in idascore_data:
    # Convert the 'Scores' list into a DataFrame
    df_idascore = pd.DataFrame(idascore_data['Scores'])
else:
    print("No IDASCORE data found or 'Scores' key missing in the response.")
    df_idascore = pd.DataFrame() # Create an empty DataFrame if no data

df_idascore.head()

                       EmbryoID  Viability  Time Version            Timestamp
0  D2019.06.19_S00028_I3253_P-1       1.03   115   2.0.4  2024.04.30 07:55:37
1  D2019.06.19_S00028_I3253_P-2       1.05   115   2.0.4  2024.04.30 07:55:35
2  D2019.06.19_S00028_I3253_P-3       1.06   115   2.0.4  2024.04.30 07:55:33
3  D2019.06.19_S00028_I3253_P-4       1.01   115   2.0.4  2024.04.30 07:56:06
4  D2019.07.03_S00054_I3253_P-2       3.93   120   2.0.4  2024.05.28 15:20:00


In [70]:
df_idascore.head()

,EmbryoID,Viability,Time,Version,Timestamp
0,D2019.06.19_S00028_I3253_P-1,1.03,115,2.0.4,2024.04.30 07:55:37
1,D2019.06.19_S00028_I3253_P-2,1.05,115,2.0.4,2024.04.30 07:55:35
2,D2019.06.19_S00028_I3253_P-3,1.06,115,2.0.4,2024.04.30 07:55:33
3,D2019.06.19_S00028_I3253_P-4,1.01,115,2.0.4,2024.04.30 07:56:06
4,D2019.07.03_S00054_I3253_P-2,3.93,120,2.0.4,2024.05.28 15:20:00


# Fluxo de um embryoscope

In [60]:
def rate_limited_request(func, *args, **kwargs):
    """Execute function with rate limiting (10 requests per second)"""
    time.sleep(0.1)  # 100ms delay to stay under 10 requests/second
    return func(*args, **kwargs)

def get_all_treatments_for_patients(df_patients, base_url, token):
    """Get all treatments for all patients"""
    all_treatments = []
    
    print(f"Processing {len(df_patients)} patients...")
    
    for idx, patient in df_patients.iterrows():
        patient_idx = patient['PatientIDx']
        print(f"Processing patient {idx+1}/{len(df_patients)}: {patient_idx}")
        
        # Get treatments with rate limiting
        treatments_data = rate_limited_request(get_treatments, base_url, token, patient_idx)

        if treatments_data and 'TreatmentList' in treatments_data:
            for treatment_name in treatments_data['TreatmentList']:
                # Create treatment record with patient reference
                treatment_record = {
                    'PatientIDx': patient_idx,
                    'TreatmentName': treatment_name
                }
                all_treatments.append(treatment_record)
    
    df_treatments = pd.DataFrame(all_treatments)
    print(f"Found {len(df_treatments)} treatments across all patients")
    return df_treatments

def get_all_embryo_data(df_treatments, base_url, token):
    """Get embryo data for all treatments and break down nested dictionaries into columns"""
    all_embryo_data = []
    
    print(f"Processing {len(df_treatments)} treatments...")
    
    for idx, treatment in df_treatments.iterrows():
        patient_idx = treatment['PatientIDx']
        treatment_name = treatment['TreatmentName']
        
        print(f"Processing treatment {idx+1}/{len(df_treatments)}: {treatment_name}")
        
        # Get embryo data with rate limiting
        embryo_data = rate_limited_request(get_embryo_data, base_url, token, patient_idx, treatment_name)
        
        if embryo_data and 'EmbryoDataList' in embryo_data:
            for embryo in embryo_data['EmbryoDataList']:
                # Start with basic embryo info
                embryo_record = {
                    'EmbryoID': embryo['EmbryoID'],
                    'PatientIDx': patient_idx,
                    'TreatmentName': treatment_name,
                    'AnnotationList': embryo.get('AnnotationList')
                }
                
                # Break down EmbryoDetails into separate columns
                if 'EmbryoDetails' in embryo and embryo['EmbryoDetails']:
                    details = embryo['EmbryoDetails']
                    embryo_record.update({
                        'InstrumentNumber': details.get('InstrumentNumber'),
                        'Position': details.get('Position'),
                        'WellNumber': details.get('WellNumber'),
                        'FertilizationTime': details.get('FertilizationTime'),
                        'EmbryoFate': details.get('EmbryoFate'),
                        'Description': details.get('Description'),
                        'EmbryoDescriptionID': details.get('EmbryoDescriptionID')
                    })
                
                # Break down Evaluation into separate columns
                if 'Evaluation' in embryo and embryo['Evaluation']:
                    evaluation = embryo['Evaluation']
                    embryo_record.update({
                        'EvaluationModel': evaluation.get('Model'),
                        'EvaluationScore': evaluation.get('Evaluation'),
                        'EvaluationUser': evaluation.get('User'),
                        'EvaluationDate': evaluation.get('EvaluationDate')
                    })
                
                all_embryo_data.append(embryo_record)
    
    df_embryo_data = pd.DataFrame(all_embryo_data)
    print(f"Found {len(df_embryo_data)} embryos across all treatments")
    return df_embryo_data


In [47]:
df_treatments = get_all_treatments_for_patients(df_patients.loc[2586:2590], base_url, token)
df_treatments

Processing 5 patients...
Processing patient 2587/5: PC1R85KM_45817.4796418750
{'TreatmentList': ['2025 - 987']}
Processing patient 2588/5: PC1R85KM_45772.5287441204
{'TreatmentList': ['2025  - 689', '2025 - 993']}
Processing patient 2589/5: PC1R85KM_45801.4523230440
{'TreatmentList': ['2025  - 881']}
Processing patient 2590/5: PC1R85KM_45818.4200686458
{'TreatmentList': ['2025-999']}
Processing patient 2591/5: PC1R85KM_45818.4773436690
{'TreatmentList': ['2025-998']}
Found 6 treatments across all patients


,PatientIDx,TreatmentName
0,PC1R85KM_45817.4796418750,2025 - 987
1,PC1R85KM_45772.5287441204,2025 - 689
2,PC1R85KM_45772.5287441204,2025 - 993
3,PC1R85KM_45801.4523230440,2025 - 881
4,PC1R85KM_45818.4200686458,2025-999
5,PC1R85KM_45818.4773436690,2025-998


In [61]:
df_embryo_data = get_all_embryo_data(df_treatments, base_url, token)
df_embryo_data

Processing 6 treatments...
Processing treatment 1/6: 2025 - 987
Processing treatment 2/6: 2025  - 689
Processing treatment 3/6: 2025 - 993
Processing treatment 4/6: 2025  - 881
Processing treatment 5/6: 2025-999
Error fetching embryo data: Expecting value: line 1 column 1 (char 0)
Processing treatment 6/6: 2025-998
Error fetching embryo data: Expecting value: line 1 column 1 (char 0)
Found 22 embryos across all treatments


,EmbryoID,PatientIDx,TreatmentName,AnnotationList,InstrumentNumber,Position,WellNumber,FertilizationTime,EmbryoFate,Description,EmbryoDescriptionID,EvaluationModel,EvaluationScore,EvaluationUser,EvaluationDate
0,D2025.06.09_S03568_I3253_P-1,PC1R85KM_45817.4796418750,2025 - 987,"[{'Name': 'PN', 'Time': 18.7, 'Value': '2', 'T...",3253,2,1,2025.06.09 11:30:00,Unknown,None,AA1,NaN,NaN,NaN,NaN
1,D2025.06.09_S03568_I3253_P-2,PC1R85KM_45817.4796418750,2025 - 987,None,3253,2,2,2025.06.09 11:30:00,Unknown,None,AA2,NaN,NaN,NaN,NaN
2,D2025.06.09_S03568_I3253_P-3,PC1R85KM_45817.4796418750,2025 - 987,None,3253,2,3,2025.06.09 11:30:00,Unknown,None,AA3,NaN,NaN,NaN,NaN
3,D2025.06.09_S03568_I3253_P-4,PC1R85KM_45817.4796418750,2025 - 987,None,3253,2,4,2025.06.09 11:30:00,Unknown,None,AA4,NaN,NaN,NaN,NaN
4,D2025.06.09_S03568_I3253_P-5,PC1R85KM_45817.4796418750,2025 - 987,"[{'Name': 'PN', 'Time': 20.2, 'Value': '2', 'T...",3253,2,5,2025.06.09 11:30:00,Unknown,None,AA5,NaN,NaN,NaN,NaN
5,D2025.06.09_S03568_I3253_P-6,PC1R85KM_45817.4796418750,2025 - 987,"[{'Name': 'PN', 'Time': 18.6, 'Value': '2', 'T...",3253,2,6,2025.06.09 11:30:00,Unknown,None,AA6,NaN,NaN,NaN,NaN
6,D2025.06.09_S03568_I3253_P-7,PC1R85KM_45817.4796418750,2025 - 987,None,3253,2,7,2025.06.09 11:30:00,Avoid,None,AA7,NaN,NaN,NaN,NaN
7,D2025.06.09_S03568_I3253_P-8,PC1R85KM_45817.4796418750,2025 - 987,"[{'Name': 'PN', 'Time': 18.7, 'Value': '2', 'T...",3253,2,8,2025.06.09 11:30:00,Unknown,None,AA8,NaN,NaN,NaN,NaN
8,D2025.06.09_S03568_I3253_P-9,PC1R85KM_45817.4796418750,2025 - 987,"[{'Name': 'PN', 'Time': 20.2, 'Value': '2', 'T...",3253,2,9,2025.06.09 11:30:00,Unknown,None,AA9,NaN,NaN,NaN,NaN
9,D2025.04.25_S03479_I3253_P-1,PC1R85KM_45772.5287441204,2025 - 689,"[{'Name': 'PN', 'Time': 16.7, 'Value': '2', 'T...",3253,2,1,2025.04.25 14:45:00,Avoid,None,AA1,NaN,NaN,NaN,NaN


In [57]:
df_embryo_data.loc[20, 'EmbryoDetails']

{'InstrumentNumber': 3253,
 'Position': 6,
 'WellNumber': 2,
 'FertilizationTime': '2025.05.24 11:50:00',
 'EmbryoFate': 'Freeze',
 'Description': 'EUPLOIDE XX',
 'EmbryoDescriptionID': 'AA2'}

In [58]:
df_embryo_data.loc[20, 'AnnotationList']

[{'Name': 'PN',
  'Time': 18.1,
  'Value': '2',
  'Timestamp': '2025.05.25 05:58:40'},
 {'Name': 'tPNf',
  'Time': 20.7,
  'Value': '45802.3573196875',
  'Timestamp': '2025.05.25 08:34:32'},
 {'Name': 't2',
  'Time': 23.5,
  'Value': '45802.4729064236',
  'Timestamp': '2025.05.25 11:20:59'},
 {'Name': 'FRAG2CAT',
  'Time': 27.9,
  'Value': '0-10%',
  'Timestamp': '2025.05.25 15:45:41'},
 {'Name': 'Nuclei2',
  'Time': 28.8,
  'Value': '2',
  'Timestamp': '2025.05.25 16:39:45'},
 {'Name': 'MN2Type',
  'Time': 28.8,
  'Value': 'Bi',
  'Timestamp': '2025.05.25 16:39:45'},
 {'Name': 't3',
  'Time': 34.2,
  'Value': '45802.9195690625',
  'Timestamp': '2025.05.25 22:04:10'},
 {'Name': 't4',
  'Time': 34.8,
  'Value': '45802.9420981713',
  'Timestamp': '2025.05.25 22:36:37'},
 {'Name': 't5',
  'Time': 46.2,
  'Value': '45803.4183484491',
  'Timestamp': '2025.05.26 10:02:25'},
 {'Name': 't8',
  'Time': 50.4,
  'Value': '45803.594766713',
  'Timestamp': '2025.05.26 14:16:27'},
 {'Name': 'tSB',
 

In [ ]:
def get_all_embryo_data(df_treatments, base_url, token):
    """Get embryo data for all treatments and break down nested dictionaries into columns"""
    all_embryo_data = []
    
    print(f"Processing {len(df_treatments)} treatments...")
    
    for idx, treatment in df_treatments.iterrows():
        patient_idx = treatment['PatientIDx']
        treatment_name = treatment['TreatmentName']
        
        print(f"Processing treatment {idx+1}/{len(df_treatments)}: {treatment_name}")
        
        # Get embryo data with rate limiting
        embryo_data = rate_limited_request(get_embryo_data, base_url, token, patient_idx, treatment_name)
        
        if embryo_data and 'EmbryoDataList' in embryo_data:
            for embryo in embryo_data['EmbryoDataList']:
                # Start with basic embryo info
                embryo_record = {
                    'EmbryoID': embryo['EmbryoID'],
                    'PatientIDx': patient_idx,
                    'TreatmentName': treatment_name,
                    'AnnotationList': embryo.get('AnnotationList')
                }
                
                # Break down EmbryoDetails into separate columns
                if 'EmbryoDetails' in embryo and embryo['EmbryoDetails']:
                    details = embryo['EmbryoDetails']
                    embryo_record.update({
                        'InstrumentNumber': details.get('InstrumentNumber'),
                        'Position': details.get('Position'),
                        'WellNumber': details.get('WellNumber'),
                        'FertilizationTime': details.get('FertilizationTime'),
                        'EmbryoFate': details.get('EmbryoFate'),
                        'Description': details.get('Description'),
                        'EmbryoDescriptionID': details.get('EmbryoDescriptionID')
                    })
                
                # Break down Evaluation into separate columns
                if 'Evaluation' in embryo and embryo['Evaluation']:
                    evaluation = embryo['Evaluation']
                    embryo_record.update({
                        'EvaluationModel': evaluation.get('Model'),
                        'EvaluationScore': evaluation.get('Evaluation'),
                        'EvaluationUser': evaluation.get('User'),
                        'EvaluationDate': evaluation.get('EvaluationDate')
                    })
                
                all_embryo_data.append(embryo_record)
    
    df_embryo_data = pd.DataFrame(all_embryo_data)
    print(f"Found {len(df_embryo_data)} embryos across all treatments")
    return df_embryo_data

In [59]:
df_embryo_data.loc[20, 'Evaluation']

{'Model': 'KIDScoreD5 v3.3',
 'Evaluation': '9.4',
 'User': 'ADMIN',
 'EvaluationDate': '2025.05.29'}

In [62]:
import os

# Create data_output folder if it doesn't exist
os.makedirs('data_output', exist_ok=True)

# Save dataframes to CSV files
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save without timestamp
df_patients.to_csv('data_output/patients.csv', index=False, quoting=1)
df_treatments.to_csv('data_output/treatments.csv', index=False, quoting=1)
df_embryo_data.to_csv('data_output/embryo_data.csv', index=False, quoting=1)

# Save with timestamp
df_patients.to_csv(f'data_output/patients_{timestamp}.csv', index=False, quoting=1)
df_treatments.to_csv(f'data_output/treatments_{timestamp}.csv', index=False, quoting=1)
df_embryo_data.to_csv(f'data_output/embryo_data_{timestamp}.csv', index=False, quoting=1)

print(f"Files saved to data_output folder with timestamp: {timestamp}")

Files saved to data_output folder with timestamp: 20250610_134610


In [ ]:
# Execute the flux
print("=== Starting Data Collection Flux ===")
print(f"Start time: {datetime.now()}")

# Step 1: We already have patients in df_patients
print(f"\nStep 1: Patients loaded - {len(df_patients)} patients")


In [ ]:

# Step 2: Get all treatments
print("\nStep 2: Collecting treatments...")
df_treatments = get_all_treatments_for_patients(df_patients, base_url, token)


In [ ]:

# Step 3: Get all embryo data
print("\nStep 3: Collecting embryo data...")
df_embryo_data = get_all_embryo_data(df_treatments, base_url, token)


In [ ]:

print(f"\n=== Data Collection Complete ===")
print(f"End time: {datetime.now()}")
print(f"Total patients: {len(df_patients)}")
print(f"Total treatments: {len(df_treatments)}")
print(f"Total embryos: {len(df_embryo_data)}")

In [ ]:
# Merge df_final_merged with df_idascore on 'EmbryoID'
df_final_merged_with_idascore = pd.merge(df_final_merged, df_idascore, on='EmbryoID', how='left')

print("Shape of df_final_merged:", df_final_merged.shape)
print("Shape of df_idascore:", df_idascore.shape)
print("Shape of df_final_merged_with_idascore:", df_final_merged_with_idascore.shape)
df_final_merged_with_idascore.head()

In [89]:
# Merge df_patients with df_treatments on 'PatientIDx'
df_merged = pd.merge(df_patients, df_treatments, on='PatientIDx', how='left')

# Merge the result with df_embryo_data on 'PatientIDx' and 'TreatmentName'
df_final_merged = pd.merge(df_merged, df_embryo_data, on=['PatientIDx', 'TreatmentName'], how='left')

# Merge df_final_merged with df_idascore on 'EmbryoID'
df_final_merged_with_idascore = pd.merge(df_final_merged, df_idascore, on='EmbryoID', how='inner')

print("Shape of df_patients:", df_patients.shape)
print("Shape of df_treatments:", df_treatments.shape)
print("Shape of df_embryo_data:", df_embryo_data.shape)
print("Shape of df_final_merged:", df_final_merged.shape)
print("Shape of df_idascore:", df_idascore.shape)
print("Shape of df_final_merged_with_idascore:", df_final_merged_with_idascore.shape)

df_final_merged_with_idascore.tail()

Shape of df_patients: (2591, 5)
Shape of df_treatments: (6, 2)
Shape of df_embryo_data: (22, 15)
Shape of df_final_merged: (2610, 19)
Shape of df_idascore: (9290, 5)
Shape of df_final_merged_with_idascore: (0, 23)


,PatientIDx,PatientID,FirstName,LastName,DateOfBirth,TreatmentName,EmbryoID,AnnotationList,InstrumentNumber,Position,...,Description,EmbryoDescriptionID,EvaluationModel,EvaluationScore,EvaluationUser,EvaluationDate,Viability,Time,Version,Timestamp


In [74]:
df_idascore

,EmbryoID,Viability,Time,Version,Timestamp
0,D2019.06.19_S00028_I3253_P-1,1.03,115,2.0.4,2024.04.30 07:55:37
1,D2019.06.19_S00028_I3253_P-2,1.05,115,2.0.4,2024.04.30 07:55:35
2,D2019.06.19_S00028_I3253_P-3,1.06,115,2.0.4,2024.04.30 07:55:33
3,D2019.06.19_S00028_I3253_P-4,1.01,115,2.0.4,2024.04.30 07:56:06
4,D2019.07.03_S00054_I3253_P-2,3.93,120,2.0.4,2024.05.28 15:20:00
...,...,...,...,...,...
9285,D2025.05.31_S03548_I3253_P-2,4.78,120,2.0.5,2025.06.05 10:48:38
9286,D2025.05.31_S03548_I3253_P-3,4.70,120,2.0.5,2025.06.05 10:48:38
9287,D2025.05.31_S03548_I3253_P-4,1.04,120,2.0.5,2025.06.05 10:48:39
9288,D2025.05.31_S03548_I3253_P-5,4.71,120,2.0.5,2025.06.05 10:48:39


In [120]:
df_embryo_data[df_embryo_data['EmbryoID'].fillna('').str.contains('D2025.05.13')]

,EmbryoID,PatientIDx,TreatmentName,AnnotationList,InstrumentNumber,Position,WellNumber,FertilizationTime,EmbryoFate,Description,EmbryoDescriptionID,EvaluationModel,EvaluationScore,EvaluationUser,EvaluationDate


In [121]:
df_embryo_data['EmbryoID'].values

array(['D2025.06.09_S03568_I3253_P-1', 'D2025.06.09_S03568_I3253_P-2',
       'D2025.06.09_S03568_I3253_P-3', 'D2025.06.09_S03568_I3253_P-4',
       'D2025.06.09_S03568_I3253_P-5', 'D2025.06.09_S03568_I3253_P-6',
       'D2025.06.09_S03568_I3253_P-7', 'D2025.06.09_S03568_I3253_P-8',
       'D2025.06.09_S03568_I3253_P-9', 'D2025.04.25_S03479_I3253_P-1',
       'D2025.06.09_S03571_I3253_P-1', 'D2025.06.09_S03571_I3253_P-2',
       'D2025.06.09_S03571_I3253_P-3', 'D2025.06.09_S03571_I3253_P-4',
       'D2025.06.09_S03571_I3253_P-5', 'D2025.06.09_S03571_I3253_P-6',
       'D2025.06.09_S03571_I3253_P-7', 'D2025.06.09_S03571_I3253_P-8',
       'D2025.06.09_S03571_I3253_P-9', 'D2025.05.24_S03537_I3253_P-1',
       'D2025.05.24_S03537_I3253_P-2', 'D2025.05.24_S03537_I3253_P-3'],
      dtype=object)

In [122]:
df_idascore['EmbryoID'].values

array(['D2019.06.19_S00028_I3253_P-1', 'D2019.06.19_S00028_I3253_P-2',
       'D2019.06.19_S00028_I3253_P-3', ...,
       'D2025.05.31_S03548_I3253_P-4', 'D2025.05.31_S03548_I3253_P-5',
       'D2025.05.31_S03548_I3253_P-6'], dtype=object)

In [109]:
df_idascore[df_idascore['EmbryoID'].fillna('').str.contains('D2025.05.24')]

,EmbryoID,Viability,Time,Version,Timestamp
